In [1]:
%reset -f
%matplotlib inline
from itertools import permutations
import matplotlib.pyplot as plt
import numpy as np

from aux import Generic
from ntwk import LIFNtwk
from disp import print_red, raster, set_font_size

import PARAMS as P_GLOBAL

# PARAMETERS AND CROSS-SIMULATION SUBROUTINES

## Parameters

In [2]:
P = Generic(
    N=500,
    W_A_OUT_OUT=P_GLOBAL.W_A_PC_PC,
)

## Network structure

### Item assignment

In [3]:
def assign_items(n, q, q_star, test=False):
    """
    Probabilistically assign item labels to n neurons.
    Exactly 4 items are considered, with hard-coded 
    combination probabilities. Generalization to N items
    is beyond the scope of this project.
    
    :param n: number of neurons
    :param q: prob. of neuron being in any given item group
    :param q_star: prob. of neuron being in any specific second
        item group, given that it is in another first item group
        
    :return: n x 4 boolean array of item assignments
    """
    
    # specify all item combinations
    item_combos = [
        [],  # row 0
        [0,], [1,], [2,], [3,],  # row 1
        [0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3],  # row 2
        [0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3],  # row 3
        [0, 1, 2, 3],  # row 4
    ]
    
    # compute probability of each combination
    
    ## p of row 1
    a = q * ((1 - q_star)**3)
    ## p of row 2
    b = q * q_star * ((1 - q_star)**2)
    ## p of row 3
    c = q * (q_star**2) * (1 - q_star)
    ## p of row 4
    d = q * (q_star**3)
    
    ## validate probabilities
    assert 4*a + 6*b + 4*c + d < 1
    
    ## full vector
    p = [
        1 - (4*a + 6*b + 4*c + d),  # row 0
        a, a, a, a,  # row 1
        b, b, b, b, b, b,  # row 2
        c, c, c, c,  # row 3
        d,  # row 4
    ]
    
    # assign labels
    labels = np.zeros((n, 4), bool)
    
    for i in range(n):
        
        # sample item assignment for i-th neuron
        item_combo = np.random.choice(item_combos, p=p)
        
        # convert to boolean
        labels[i, item_combo] = True
    
    if not test:
        
        return labels
    
    else:
        
        # estimate q
        q_hat = labels.mean()
        
        # estimate q_star
        q_star_hat = np.array([
            labels[labels[:, i], :][:, j].mean()
            for i, j in permutations(range(4), 2)
        ])
        
        return Generic(
            labels=labels, q_hat=q_hat, q_star_hat=q_star_hat)

#### Test

In [4]:
def test_assign_items_no_overlap():
    print_red('\n>> test_assign_items_no_overlap...\n'.upper())
    
    n = 10000
    q = 0.2
    q_star = 0.0
    
    rslt = assign_items(n=n, q=q, q_star=q_star, test=True)
    
    # ensure correct item probability  
    print_red('    q = {}'.format(q))
    print_red('    q_hat = {0:.4f}\n'.format(rslt.q_hat))

    # ensure no overlap
    print_red('    q* = {}'.format(q_star))
    print_red('    q*_hat = {0:.4f}\n'.format(rslt.q_star_hat.mean()))
    
    # ensure only 0 or 1 item per neuron
    print_red('    unique items per neuron ([0, 1]) = {}\n'.format(set(rslt.labels.sum(1))))
    

def test_assign_items_complete_overlap():
    print_red('\n>> test_assign_items_complete_overlap...\n'.upper())
    
    n = 10000
    q = 0.2
    q_star = 1.0
    
    rslt = assign_items(n=n, q=q, q_star=q_star, test=True)
    
    # ensure correct item probability  
    print_red('    q = {}'.format(q))
    print_red('    q_hat = {0:.4f}\n'.format(rslt.q_hat))

    # ensure complete overlap
    print_red('    q* = {}'.format(q_star))
    print_red('    q*_hat = {0:.4f}\n'.format(rslt.q_star_hat.mean()))
    
    # ensure only 0 or 4 items per neuron
    print_red('    unique items per neuron ([0, 4]) = {}\n'.format(set(rslt.labels.sum(1))))
    

def test_assign_items_partial_overlap():
    print_red('\n>> test_assign_items_partial_overlap...\n'.upper())
    
    n = 10000
    q = 0.2
    q_star = 0.5
    
    rslt = assign_items(n=n, q=q, q_star=q_star, test=True)
    
    # ensure correct item probability  
    print_red('    q = {}'.format(q))
    print_red('    q_hat = {0:.4f}\n'.format(rslt.q_hat))

    # ensure correct partial overlap
    print_red('    q* = {}'.format(q_star))
    print_red('    q*_hat = {0:.4f}\n'.format(rslt.q_star_hat.mean()))
    
    # ensure any number of items per neuron
    print_red('    unique items per neuron ([0, 1, 2, 3, 4]) = {}\n'.format(set(rslt.labels.sum(1))))
            

np.random.seed(0)

test_assign_items_no_overlap()
test_assign_items_complete_overlap()
test_assign_items_partial_overlap()


>> TEST_ASSIGN_ITEMS_NO_OVERLAP...

    q = 0.2
    q_hat = 0.1985

    q* = 0.0
    q*_hat = 0.0000

    unique items per neuron ([0, 1]) = {0, 1}


>> TEST_ASSIGN_ITEMS_COMPLETE_OVERLAP...

    q = 0.2
    q_hat = 0.2014

    q* = 1.0
    q*_hat = 1.0000

    unique items per neuron ([0, 4]) = {0, 4}


>> TEST_ASSIGN_ITEMS_PARTIAL_OVERLAP...

    q = 0.2
    q_hat = 0.2005

    q* = 0.5
    q*_hat = 0.5047

    unique items per neuron ([0, 1, 2, 3, 4]) = {0, 1, 2, 3, 4}



Tests passed if:
1. Estimates (q_hat, q_star_hat) are near truth values.
1. If unique items per neuron match parentheticals.

### Weight matrix

In [5]:
def make_w(labels, p_0, p_1, P, test=False):
    """Create weight matrix given neuron labeling."""
    
    n = len(labels)
    
    # get all intra-item cxn possibilities
    mask_intras = []
    
    for i in range(4):
        
        # get intra-cxns for item i
        mask_intra_ = np.outer(labels[:, i], labels[:, i])
        
        # zero self-cxns
        np.fill_diagonal(mask_intra_, 0)
        
        mask_intras.append(mask_intra_)
    
    # take union of intra-item cxn possibilities
    mask_intra = np.logical_or.reduce(mask_intras)
    n_intra = mask_intra.sum()
    
    # build cxn matrix
    cxn = np.zeros((n, n), bool)
    
    # set intra-item cnxs
    assert 0 <= p_1 <= 1
    cxn[mask_intra] = np.random.binomial(1, p_1, n_intra)
    
    # set non-intra cxns
    p_2 = ((n * (n - 1) * p_0) - (n_intra * p_1)) / ((n * (n - 1)) - n_intra)
    
    assert 0 <= p_2 <= 1
    cxn[~mask_intra] = np.random.binomial(1, p_2, (~mask_intra).sum())
    
    # zero self-cxns
    np.fill_diagonal(cxn, 0)
    
    # convert to weight matrix
    w = cxn * P.W_A_OUT_OUT
    
    if not test:
        
        return w
    
    else:
        
        # estimate p_0
        mask_non_self = ~np.eye(n, dtype=bool)
        p_0_hat = (w > 0)[mask_non_self].mean()
        
        # estimate p_1
        p_1_hat = np.array([(w > 0)[mask_intra].mean() for mask_intra in mask_intras])
        
        return Generic(w=w, p_0_hat=p_0_hat, p_1_hat=p_1_hat)

#### Test

In [6]:
def test_make_w_no_overlap():
    print_red('\n>> test_make_w...\n'.upper())
    
    n = 10000
    q = 0.2
    q_star = 0
    
    p_0 = 0.1
    p_1s = [0, 0.1, 0.2]
    
    labels = assign_items(n=n, q=q, q_star=q_star)
    
    for p_1 in p_1s:
        
        rslt = make_w(labels, p_0, p_1, P, test=True)
        
        print_red('    p_0 = {}'.format(p_0))
        print_red('    p_0_hat = {0:.4f}\n'.format(rslt.p_0_hat))
        
        print_red('    p_1 = {}'.format(p_1))
        print_red('    p_1_hat = {0:.4f}\n'.format(rslt.p_1_hat.mean()))
        

def test_make_w_partial_overlap():
    print_red('\n>> test_make_w_partial_overlap...\n'.upper())
    
    n = 10000
    q = 0.2
    q_star = 0.5
    
    p_0 = 0.1
    p_1s = [0, 0.1, 0.2]
    
    labels = assign_items(n=n, q=q, q_star=q_star)
    
    for p_1 in p_1s:
        
        rslt = make_w(labels, p_0, p_1, P, test=True)
        
        print_red('    p_0 = {}'.format(p_0))
        print_red('    p_0_hat = {0:.4f}\n'.format(rslt.p_0_hat))
        
        print_red('    p_1 = {}'.format(p_1))
        print_red('    p_1_hat = {0:.4f}\n'.format(rslt.p_1_hat.mean()))
        

test_make_w_no_overlap()
test_make_w_partial_overlap()


>> TEST_MAKE_W...

    p_0 = 0.1
    p_0_hat = 0.1000

    p_1 = 0
    p_1_hat = 0.0000

    p_0 = 0.1
    p_0_hat = 0.1000

    p_1 = 0.1
    p_1_hat = 0.0999

    p_0 = 0.1
    p_0_hat = 0.1000

    p_1 = 0.2
    p_1_hat = 0.2000


>> TEST_MAKE_W_PARTIAL_OVERLAP...

    p_0 = 0.1
    p_0_hat = 0.1000

    p_1 = 0
    p_1_hat = 0.0000

    p_0 = 0.1
    p_0_hat = 0.1000

    p_1 = 0.1
    p_1_hat = 0.0999

    p_0 = 0.1
    p_0_hat = 0.1000

    p_1 = 0.2
    p_1_hat = 0.1999



Tests passed if:
1. All estimates (*_hat) approximate truth values.

### Top-level build

In [ ]:
def make_ntwk(n, q, q_star, p_0, p_1, P):
    ntwk.order = ...
    ntwk.labels = ...
    return ntwk

## Stimulation paradigm

In [ ]:
def make_spks_up(ntwk, items, P):
    return spks_up

## Recall accuracy

In [ ]:
def calc_acc(rsp, items, P):
    return acc

# SIMULATIONS

## Examples

### Simulation 

In [ ]:
def xmpl(q_star, p_0, p_1, items):
    
    # build ntwk and run smln
    ntwk = make_ntwk(n=N, q=Q, q_star=q_star, p_0=p_0, p_1=p_1, P=P)
    spks_up = make_spks_up(ntwk, items, P)
    rsp = ntwk.run(spks_up, P.DT)
    acc = calc_acc(rsp, items, P)

    # show raster and CTL->OUT weights
    fig, axs = plt.subplots(2, 1, figsize=(15, 8), tight_layout=True)

    ## raster
    raster(axs[0], rsp.ts, rsp.spks, order=ntwk.order)

    ## CTL->OUT weights
    w_out_ctl_items = ...
    w_out_ctl_all = ...

    axs[1].plot(rsp.ts, w_out_ctl_all, color='k', lw=2)
    axs[1].plot(rsp.ts, w_out_ctl_items, color='r', lw=2)
    
    axs[0].set_title('ACC = {0:.4f}'.format(acc))
    for ax in axs:
        set_font_size(ax, 16)
        
    return Generic(
        ntwk=ntwk,
        items=items,
        spks_up=spks_up,
        rsp=rsp,
        acc=acc,
        fig=fig,
        ax=axs)

### Single-item, no overlap

In [ ]:
xmpl(q_star=0, p_0=0.1, p_1=0.2, items=[1])

### Single-item, modest overlap

In [ ]:
xmpl(q_star=0.2, p_0=0.1, p_1=0.2, items=[1])

### Multi-item, no overlap

In [ ]:
xmpl(q_star=0, p_0=0.1, p_1=0.2, items=[1, 3])

### Multi-item, modest overlap

In [ ]:
xmpl(q_star=0.2, p_0=0.1, p_1=0.2, items=[1, 3])

## Parameter dependence

### Simulation

In [ ]:
def param_variation(q_stars, p_0s, p_1s):
    
    return Generic(
        ...
    )

### Varying interconnectivity (p_0) and overlap (q_star)

### Varying intraconnectivity (p_1) and overlap (q_star)